In [1]:
import pandas as pd
import numpy as np
from read_path_module import read_data_relative_path
from write_path_module import write_data_relative_path

## Importing 2003 - 2006 Data

In [2]:
# Read in txt file
df = read_data_relative_path('./data/CDC_Natality/2003_2006.txt', data_type='txt_tab')

# Write to csv for more convenient future use
write_data_relative_path(df, './data/CDC_Natality/2003_2006.csv', data_type='csv')
del df

# Import in data and rename
df_2003_2006 = read_data_relative_path('./data/CDC_Natality/2003_2006.csv', data_type='csv')
df_2003_2006.sample(3)

,Notes,Year,Year Code,Month,Month Code,State,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age
164,NaN,2003.0,2003.0,January,1.0,Illinois,17.0,"LaSalle County, IL",17099.0,118.0,27.03,38.83
5310,Total,2003.0,2003.0,September,9.0,Nebraska,31.0,NaN,NaN,2267.0,27.34,38.72
3688,NaN,2003.0,2003.0,June,6.0,Utah,49.0,"Weber County, UT",49057.0,331.0,26.20,38.15


## Importing 2007 - 2018 Data

In [3]:
# Create path list for all txt files
lst_paths = ['./data/CDC_Natality/'+str(i)+'.txt' for i in range(2007,2018+1)]

# Read in first csv
df = read_data_relative_path('./data/CDC_Natality/2007.txt', data_type='txt_tab').head(1)

# Delete data to create empty, but properly columned dataframe
df = df[0:0]

# Iterate through path names and append them to empty dataframe above
for path in lst_paths:
    temp = read_data_relative_path(path, data_type='txt_tab')
    df = pd.concat([df, temp])
    del temp

# Write to csv for more convenient future use
write_data_relative_path(df, './data/CDC_Natality/2007_2018.csv', data_type='csv')
del df

# Import in data and rename
df_2007_2018 = read_data_relative_path('./data/CDC_Natality/2007_2018.csv', data_type='csv')
df_2007_2018.sample(5)

,Notes,Year,Year Code,Month,Month Code,State,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age,Average OE Gestational Age
29554,NaN,2010.0,2010.0,August,8.0,Florida,12.0,"Santa Rosa County, FL",12113.0,156,27.24,38.49,38.19
91565,NaN,2018.0,2018.0,February,2.0,Ohio,39.0,"Delaware County, OH",39041.0,146,30.51,38.61,38.59
43753,NaN,2012.0,2012.0,April,4.0,Texas,48.0,"Wichita County, TX",48485.0,145,25.32,38.53,38.52
54506,NaN,2013.0,2013.0,August,8.0,North Carolina,37.0,"Cabarrus County, NC",37025.0,219,28.37,38.70,38.83
90677,NaN,2018.0,2018.0,January,1.0,Louisiana,22.0,"Terrebonne Parish, LA",22109.0,139,26.81,38.44,38.19


## Combine the two datasets

In [4]:
# Check to see if column names match
col_2003 = df_2003_2006.columns.to_list()
col_2007 = df_2007_2018.columns.to_list()

def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

print('The difference between the two datasets is the column: ', Diff(col_2007, col_2003)) 
# print('The difference between the two datasets is the column: ', Diff(col_2003, col_2007)) # No difference

# Drop this column from the 2007 DF
df_2007_2018.drop(columns=['Average OE Gestational Age'], inplace=True)

# Concatenate the two columns
df = pd.concat([df_2003_2006, df_2007_2018])
df.sample(5)

The difference between the two datasets is the column:  ['Average OE Gestational Age']


,Notes,Year,Year Code,Month,Month Code,State,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age
35682,NaN,2011.0,2011.0,May,5.0,Colorado,8.0,"El Paso County, CO",8041.0,788,27.46,38.65
46853,Total,2012.0,2012.0,September,9.0,Michigan,26.0,NaN,NaN,9397,27.64,38.74
15877,NaN,2005.0,2005.0,February,2.0,Michigan,26.0,"Oakland County, MI",26125.0,1128,29.69,38.81
15034,NaN,2005.0,2005.0,January,1.0,California,6.0,"San Mateo County, CA",6081.0,746,30.47,39.06
5607,NaN,2003.0,2003.0,September,9.0,Wisconsin,55.0,"Sheboygan County, WI",55117.0,132,27.2,38.83


In [5]:
#There are some rows that:
    # are footnotes about the dataset (i.e. everything is null besides the note column)
    # are totals of each year, month, and state grouping
# Here we will remove those rows
# Before that we will check to make sure we are just removing those ones
    
# number of rows of full dataset
num_total = df.shape[0]

# number of rows where everything is null besides note column (the pesky footnote rows)
num_all_null = df[df.iloc[:, -11:].isnull().all(axis=1)].shape[0]

# number of rows where note is null (this is a valid entry)
num_note_null = df[df['Notes'].isna()].shape[0]

# number of rows where note is 'Total' (this is a valid entry)
num_note_total = df[df['Notes']=='Total'].shape[0]

# sum up these row counts and see if they match
print(num_total, num_all_null + num_note_null + num_note_total, num_all_null, num_note_null, num_note_total)

# since they do we will concatenate everything except for the pesky footnote rows
df = df[df['Notes'].isna()]

128638 128638 737 117888 10013


## - Change the 'Missing County' label to NA

In [6]:
col_lst = df.columns.to_list()

for col in col_lst:
    df.loc[df[col] == 'Missing County', col] = np.nan
del col_lst

/Users/michaellink/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


## - Convert appropriate columns to int

In [7]:
# df['Month Code'] = df['Month Code'].astype(int)
col_lst = ['Year', 'Year Code', 'Month Code', 'State Code', 'County Code']

for col in col_lst:
    df[col] = df[col].astype(int)

### - Rename columns

In [8]:
df.rename(columns = {
    "Year": "Birth Year",
    "Month": "Birth Month",
    "Month Code": "Birth Month Code"},
         inplace = True)

### - Remove NA rows where appropriate

In [9]:
print('There were ',df['Births'].isna().sum(), ' Births NA rows.')

# Remove rows where birth data is not present
df = df[-df['Births'].isna()]

print( 'There are now ', df['Births'].isna().sum(), ' Births NA rows.')

There were  4824  Births NA rows.
There are now  0  Births NA rows.


### - Create pandas datetime column

In [10]:
df['Birth Date'] = pd.to_datetime(df['Birth Year'].astype(str) + '-' + df['Birth Month Code'].astype(str) + '-' + '15', format='%Y-%m-%d')
df.head(2)

,Notes,Birth Year,Year Code,Birth Month,Birth Month Code,State,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age,Birth Date
0,NaN,2003,2003,January,1,Alabama,1,"Baldwin County, AL",1003,131,26.77,37.97,2003-01-15
1,NaN,2003,2003,January,1,Alabama,1,"Calhoun County, AL",1015,124,25.32,38.63,2003-01-15


### - Backtrack to calculate conception date
- At a high level
    - A menstrual cycle starts
    - Two weeks later ovulation occurs
    - Sparks fly and conception occurs
    - ~38 weeks later parents sleep cycles are forever changed...
    
    
- Within the dataset LMP gestational age is given. This is the estimate of the last menstrual cycle


- To calculate conception date we:
    - Take date of birth (+/- 15 days) then subtract LMP Age then add 2 weeks to ovulation

In [11]:
# Transforming the gestational age into a timedelta for time arithmetic
lst_time_intrval = [pd.Timedelta(float(x)*7, unit='days') for x in df['Average LMP Gestational Age']]
    
# Date of birth - LMP Age + 2 weeks to ovulation
df['Conception Date'] = df['Birth Date'] - pd.Series(lst_time_intrval).values + pd.Timedelta(2, unit='W')

# Standard 38 weeks
df['Conception Date Standard'] = df['Birth Date'] - pd.Timedelta(38, unit='W')

df[['Birth Date','Conception Date', 'Conception Date Standard']].head(2)

,Birth Date,Conception Date,Conception Date Standard
0,2003-01-15,2002-05-08 05:02:24,2002-04-24
1,2003-01-15,2002-05-03 14:09:36,2002-04-24


### - Create Conception Year, Month, and Month Code Columns

In [12]:
df['Conception Year'] = pd.DatetimeIndex(df['Conception Date']).year

df['Conception Month'] = pd.DatetimeIndex(df['Conception Date']).month_name()

df['Conception Month Code'] = pd.DatetimeIndex(df['Conception Date']).month

df['Conception Year Standard'] = pd.DatetimeIndex(df['Conception Date Standard']).year

df['Conception Month Standard'] = pd.DatetimeIndex(df['Conception Date Standard']).month_name()

df['Conception Month Code Standard'] = pd.DatetimeIndex(df['Conception Date Standard']).month


# Convert to integers
col_lst = ['Conception Year', 'Conception Month Code', 'Conception Year Standard', 'Conception Month Code Standard']

for col in col_lst:
    df[col] = df[col].astype(int)

### - Clean, reorder, and create new columns

In [13]:
df['State Acronym'] = df['County'].str[-2:]
df[['County','temp']] = df['County'].str.split(',', expand=True)
df.drop(columns=['temp'], inplace=True)

df.sample(3)

df.columns.to_list()

col_lst = ['Birth Year', 'Conception Year', 'Conception Year Standard', 'Birth Month', 'Conception Month',
           'Conception Month Standard', 'Birth Month Code', 'Conception Month Code', 'Conception Month Code Standard',
           'Birth Date', 'Conception Date', 'Conception Date Standard', 'State', 'State Acronym',
           'State Code','County','County Code', 'Births','Average Age of Mother',
           'Average LMP Gestational Age','Notes']

df = df[col_lst]
del col_lst
print(df.shape)
df.sample(3)

(113064, 21)


,Birth Year,Conception Year,Conception Year Standard,Birth Month,Conception Month,Conception Month Standard,Birth Month Code,Conception Month Code,Conception Month Code Standard,Birth Date,...,Conception Date Standard,State,State Acronym,State Code,County,County Code,Births,Average Age of Mother,Average LMP Gestational Age,Notes
28094,2006,2006,2006,October,February,January,10,2,1,2006-10-15,...,2006-01-22,Alabama,AL,1,Unidentified Counties,1999,2384,25.18,38.26,NaN
52887,2013,2012,2012,June,October,September,6,10,9,2013-06-15,...,2012-09-22,Hawaii,HI,15,Honolulu County,15003,1100,28.92,38.51,NaN
14194,2008,2007,2007,September,December,December,9,12,12,2008-09-15,...,2007-12-24,Rhode Island,RI,44,Washington County,44009,90,30.28,38.90,NaN


## - Write result to csv

In [14]:
# Write to csv for more convenient future use
write_data_relative_path(df, './data/CDC_Natality/2003_2018.csv', data_type='csv')
# del df